In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# KNN

## Imports for K-NN

In [ ]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier

In [ ]:
pip install dmba

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 43.3 MB/s eta 0:00:00


In [ ]:
from dmba import classificationSummary

Colab environment detected.


In [ ]:
df = pd.read_excel('/content/for_machine_learningdf.xlsx', index_col='customerID')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5042 entries, 7590-VHVEG to 2775-SEFEE
Data columns (total 31 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   seniorCitizen                          5042 non-null   int64  
 1   partner                                5042 non-null   int64  
 2   dependents                             5042 non-null   int64  
 3   tenure                                 5042 non-null   int64  
 4   phoneService                           5042 non-null   int64  
 5   paperlessBilling                       5042 non-null   int64  
 6   monthlyCharges                         5042 non-null   float64
 7   totalCharges                           5042 non-null   float64
 8   gender_Male                            5042 non-null   int64  
 9   multipleLines_True                     5042 non-null   int64  
 10  multipleLines_No phone service         5042 non-null   int64  

# Splitting Data into Training and Validation Data

In [ ]:
trainData, validData = train_test_split(df.drop('partner', axis=1), test_size=0.1, random_state=0)

In [ ]:
df = df.drop('partner', axis=1)

In [ ]:
df.paperlessBilling.value_counts()

,count
paperlessBilling,
1,2995
0,2047


In [ ]:
df.phoneService.value_counts()

,count
phoneService,
1,4553
0,489


In [ ]:
continuous = ['tenure', 'monthlyCharges', 'totalCharges']

# Standardizing the Continuous features of the training data

In [ ]:
scaler = preprocessing.StandardScaler()

In [ ]:
scaler.fit(trainData[continuous])

StandardScaler()

In [ ]:
categorical = [col for col in df.columns if col not in continuous]

In [ ]:
dfNorm = pd.concat([pd.DataFrame(scaler.transform(df[continuous]),
             columns=['ztenure', 'zphoneService', 'zpaperlessBilling']),
           df[categorical].reset_index()], axis=1).set_index('customerID')

In [ ]:
pd.set_option('display.max_columns',len(dfNorm.columns)+1)

In [ ]:
dfNorm.head()

,ztenure,zphoneService,zpaperlessBilling,seniorCitizen,dependents,phoneService,paperlessBilling,gender_Male,multipleLines_True,multipleLines_No phone service,internetService_DSL,internetService_Fiber optic,onlineSecurity_True,onlineSecurity_No internet service,onlineBackup_True,onlineBackup_No internet service,deviceProtection_True,deviceProtection_No internet service,techSupport_True,techSupport_No internet service,streamingTV_True,streamingTV_No internet service,streamingMovies_True,streamingMovies_No internet service,contract_One year,contract_Two year,paymentMethod_Credit card (automatic),paymentMethod_Electronic check,paymentMethod_Mailed check,churn
customerID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
7590-VHVEG,-1.291446,-1.169449,-1.002607,0,0,0,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
5575-GNVDE,0.055517,-0.268591,-0.180983,0,0,1,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0
3668-QPYBK,-1.250629,-0.371641,-0.968013,0,0,1,1,1,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
7795-CFOCW,0.504505,-0.755586,-0.202521,0,0,0,0,1,0,1,1,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0
9237-HQITU,-1.250629,0.188486,-0.948794,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1


In [ ]:
trainNorm = dfNorm.loc[trainData.index]

In [ ]:
validNorm = dfNorm.loc[validData.index]

# Separating the response from training and validation

In [ ]:
train_X = trainNorm.drop('churn', axis=1)

In [ ]:
train_y = trainNorm.churn

In [ ]:
valid_X = validNorm.drop('churn', axis=1)

In [ ]:
valid_y = validNorm.churn

## Determining the best value of K

In [ ]:
results = []

for k in range(1, 15):
  knn = KNeighborsClassifier(n_neighbors=k).fit(train_X, train_y)
  results.append({
      'k':k,
      'accuracy': accuracy_score(valid_y, knn.predict(valid_X))
  })

In [ ]:
pd.DataFrame(results)

,k,accuracy
0,1,0.740594
1,2,0.772277
2,3,0.766337
3,4,0.782178
4,5,0.794059
5,6,0.788119
6,7,0.794059
7,8,0.790099
8,9,0.790099
9,10,0.788119


# Seems that K=7 if strictly determining best K on accuracy, need to adjust though because want best K based on high TPR and low FNR.

## Best value of K if optimizing recall

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import recall_score

In [ ]:
results = []
params = {
    'weights': ['uniform', 'distance']
}

for k in range(1, 15):
  knn = KNeighborsClassifier(n_neighbors=k)
  knn_grid = GridSearchCV(
      estimator=knn,
      param_grid=params,
      scoring='recall',
      cv=5
  )

  knn_grid.fit(train_X, train_y)

  results.append({
      'k': k,
      'best_param': knn_grid.best_params_['weights'],
      'recall_valid': recall_score(valid_y, knn_grid.predict(valid_X))

  })

In [ ]:
pd.DataFrame(results)

,k,best_param,recall_valid
0,1,uniform,0.510638
1,2,distance,0.503546
2,3,uniform,0.546099
3,4,distance,0.496454
4,5,uniform,0.595745
5,6,distance,0.553191
6,7,uniform,0.560284
7,8,distance,0.531915
8,9,uniform,0.574468
9,10,distance,0.546099


## Best K is 5 to optimize recall on validation data is 5, with uniform weights. Does not compare to best classification accuracy in best Logistic Regression model.